# Random Forest: Hyperparameter Tuning

## Packages and Presets

In [1]:
import pandas as pd
import numpy as np
import yaml
import os
import joblib

from sklearn.metrics import (
    f1_score, 
    balanced_accuracy_score,
)

from sklearn.preprocessing import StandardScaler, OneHotEncoder

from sklearn.model_selection import StratifiedKFold

import matplotlib.pyplot as plt
import seaborn as sns 

from imblearn.over_sampling import SMOTE

from sklearn.ensemble import RandomForestClassifier

from tsfresh.transformers.per_column_imputer import PerColumnImputer
from tsfresh.feature_extraction.settings import EfficientFCParameters, ComprehensiveFCParameters
from tsfresh.feature_extraction import extract_features

from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

pd.set_option('display.max_columns', None)
%load_ext blackcellmagic
%load_ext autoreload

import optuna
import warnings

In [3]:
NUM_TRIALS = 500

with open("classical_ml_config.yaml", "r") as file:
    config = yaml.safe_load(file)

## No Feature Engineering

In [4]:
# Load data
train_df = pd.read_csv(config["paths"]["ptb_train"], header=None)

In [29]:
X_train_all = train_df.iloc[:, :-1]
y_train_all = train_df.iloc[:, -1]

In [5]:
smote = SMOTE(random_state=config["general"]["seed"])
X_train_all, y_train_all = smote.fit_resample(X_train_all, y_train_all)

In [6]:
# create objective function for optuna
def objective_no_feat_eng(trial):

    # see: https://medium.com/@ethannabatchian/optimizing-random-forest-models-a-deep-dive-into-hyperparameter-tuning-with-optuna-b8e4fe7f3670
    hyperparams = {
        "n_estimators": trial.suggest_int("n_estimators", 100, 1000),
        "max_depth": trial.suggest_int("max_depth", 10, 50),
        "min_samples_split": trial.suggest_int("min_samples_split", 2, 32),
        "min_samples_leaf": trial.suggest_int("min_samples_leaf", 1, 32),
        "criterion": trial.suggest_categorical("criterion", ["gini", "entropy", "log_loss"]),
        "bootstrap": trial.suggest_categorical("bootstrap", [True, False]),
    }
    
    
    f1_scores = []
    
    skf = StratifiedKFold(n_splits=5, random_state=config["general"]["seed"], shuffle=True)
    for fold_num, (train_idx, val_idx) in enumerate(skf.split(X_train_all, y_train_all)):
        X_train, X_val = X_train_all.iloc[train_idx], X_train_all.iloc[val_idx]
        y_train, y_val = y_train_all[train_idx], y_train_all[val_idx]
        
        
        rf = RandomForestClassifier(
            random_state=config["general"]["seed"],
            n_jobs=-1,
            **hyperparams
        )
        rf.fit(X_train, y_train)
        
        y_preds = rf.predict(X_val)
        
        score = f1_score(y_val, y_preds)
        f1_scores.append(score)
            
        trial.report(np.mean(f1_scores), fold_num)
        if trial.should_prune():
            raise optuna.TrialPruned()
                    
        
    return np.mean(f1_scores)

In [7]:
# prune bad trials 
pruner = optuna.pruners.MedianPruner(n_startup_trials=20, n_warmup_steps=2)

study_no_feat_eng = optuna.create_study(
    direction="maximize",
    study_name="rf_no_feat_eng",
    sampler=optuna.samplers.TPESampler(seed=config["general"]["seed"]),
    pruner=pruner,
)

[I 2024-05-10 12:07:16,489] A new study created in memory with name: rf_no_feat_eng


In [8]:
study_no_feat_eng.optimize(
    objective_no_feat_eng, 
    n_trials=NUM_TRIALS,
    timeout = 20 * 60 * 60, # timeout after 20 hours
    show_progress_bar=True
)

  0%|          | 0/500 [00:00<?, ?it/s]

[I 2024-05-10 12:07:23,162] Trial 0 finished with value: 0.9532119724290524 and parameters: {'n_estimators': 437, 'max_depth': 48, 'min_samples_split': 24, 'min_samples_leaf': 20, 'criterion': 'gini'}. Best is trial 0 with value: 0.9532119724290524.
[I 2024-05-10 12:07:37,891] Trial 1 finished with value: 0.972841631428144 and parameters: {'n_estimators': 880, 'max_depth': 34, 'min_samples_split': 23, 'min_samples_leaf': 1, 'criterion': 'gini'}. Best is trial 1 with value: 0.972841631428144.
[I 2024-05-10 12:07:42,607] Trial 2 finished with value: 0.9597617704996864 and parameters: {'n_estimators': 263, 'max_depth': 17, 'min_samples_split': 11, 'min_samples_leaf': 17, 'criterion': 'log_loss'}. Best is trial 1 with value: 0.972841631428144.
[I 2024-05-10 12:07:45,974] Trial 3 finished with value: 0.9583604988987255 and parameters: {'n_estimators': 225, 'max_depth': 21, 'min_samples_split': 13, 'min_samples_leaf': 15, 'criterion': 'gini'}. Best is trial 1 with value: 0.972841631428144.
[

In [9]:
best_params = study_no_feat_eng.best_params
    
print(best_params)

if "random_forest_no_feat_eng" in config:
    config["random_forest_no_feat_eng"].update(best_params)
else:
    config["random_forest_no_feat_eng"] = best_params

# see: https://stackoverflow.com/questions/12470665/how-can-i-write-data-in-yaml-format-in-a-file
with open("classical_ml_config.yaml", "w") as file:
    yaml.dump(config, file, default_flow_style=False)

{'n_estimators': 391, 'max_depth': 25, 'min_samples_split': 3, 'min_samples_leaf': 1, 'criterion': 'entropy'}


In [10]:
p_importance_no_feat_eng = optuna.visualization.plot_param_importances(study_no_feat_eng)
p_importance_no_feat_eng.show()

In [11]:
p_history_no_feat_eng = optuna.visualization.plot_optimization_history(study_no_feat_eng)
p_history_no_feat_eng.show()

## With Feature Engineering

In [18]:
# Load data
train_df = pd.read_csv(config["paths"]["ptb_train"], header=None)

In [19]:
X_train_all = train_df.iloc[:, :-1]
y_train_all = train_df.iloc[:, -1]

In [20]:
X_train_all.columns = X_train_all.columns.astype(str)

Note that for reasons of time, we will not carry out feature engineering for every validation split separately. Instead, we will use the same feature engineering for all validation splits. This is not ideal, but it is a compromise that we have to make in order to keep the runtime of this notebook within reasonable limits.

In [21]:
X_train_all["id"] = X_train_all.index
X_train_melted =(
    X_train_all
    .melt(id_vars="id", var_name="time", value_name="value")
    .sort_values(by=["id", "time"])
)

# get rid of padding to not ruin the engineered features
# for simplicity, we drop all 0s, as only few "true" 0s are in the data
X_train_melted["value"] = X_train_melted["value"].replace(0, np.nan)
X_train_melted = X_train_melted.dropna()

In [22]:
# We will follow this tutorial:
# https://towardsdatascience.com/expanding-your-regression-repertoire-with-regularisation-903d2c9f7b28
# and will use the EfficientFCParameters
# for feature extraction
X_train_augmented = extract_features(
    X_train_melted,
    column_id="id",
    column_sort="time",
    column_value="value",
    default_fc_parameters=EfficientFCParameters(),
)  

X_train_merged = pd.merge(
    X_train_all, X_train_augmented, left_index=True, right_index=True
)

# assert that no rows were lost
assert X_train_merged.shape[0] == X_train_all.shape[0]
assert X_train_merged.index.equals(X_train_all.index)

Feature Extraction:  10%|█         | 8/80 [00:09<00:36,  1.96it/s]

In [ ]:
imputer = PerColumnImputer()
X_train_merged = imputer.fit_transform(X_train_merged)

Next, we create a Pipeline to standardize and one-hot encode the data. This is necessary since some of the newly created features are categorical or do not lie within the same range as the original features.

In [ ]:
cat_cols = X_train_merged.loc[:, X_train_merged.nunique() < 5].columns
numeric_cols = X_train_merged.loc[:, X_train_merged.nunique() >= 5].columns

# preprocessing pipeline for numerical features
num_trans = Pipeline(steps=[
    ("scaler", StandardScaler())
])

# preprocessing pipeline for categorical features
cat_trans = Pipeline(steps=[
    ('encoder', OneHotEncoder(handle_unknown="ignore"))
])


preprocessor = ColumnTransformer(transformers=[
    ("num", num_trans, numeric_cols),
    ("cat", cat_trans, cat_cols)
])

In [ ]:
# create objective function for optuna
def objective_feat_eng(trial):
    
    # see: https://medium.com/@ethannabatchian/optimizing-random-forest-models-a-deep-dive-into-hyperparameter-tuning-with-optuna-b8e4fe7f3670
    hyperparams = {
        "n_estimators": trial.suggest_int("n_estimators", 100, 1000),
        "max_depth": trial.suggest_int("max_depth", 10, 50),
        "min_samples_split": trial.suggest_int("min_samples_split", 2, 32),
        "min_samples_leaf": trial.suggest_int("min_samples_leaf", 1, 32),
        "criterion": trial.suggest_categorical("criterion", ["gini", "entropy", "log_loss"]),
        "bootstrap": trial.suggest_categorical("bootstrap", [True, False]),
    }
    
    
    
    f1_scores = []
    
    skf = StratifiedKFold(n_splits=5, random_state=config["general"]["seed"], shuffle=True)
    for fold_num, (train_idx, val_idx) in enumerate(skf.split(X_train_merged, y_train_all)):
        X_train, X_val = X_train_merged.iloc[train_idx], X_train_merged.iloc[val_idx]
        y_train, y_val = y_train_all[train_idx], y_train_all[val_idx] 
        
        smote = SMOTE(random_state=config["general"]["seed"])
        X_train, y_train = smote.fit_resample(X_train, y_train) 
        
        X_train = preprocessor.fit_transform(X_train)
        X_val = preprocessor.transform(X_val)
        
        rf = RandomForestClassifier(
            random_state=config["general"]["seed"],
            n_jobs=-1,
            **hyperparams
        )
        rf.fit(X_train, y_train)
        
        y_preds = rf.predict(X_val)
        
        score = f1_score(y_val, y_preds)
        f1_scores.append(score)
            
        trial.report(np.mean(f1_scores), fold_num)
        if trial.should_prune():
            raise optuna.TrialPruned()
        
            
        
    return np.mean(score)

In [ ]:
# prune bad trials 
pruner = optuna.pruners.MedianPruner(n_startup_trials=20, n_warmup_steps=2)

study_feat_eng = optuna.create_study(
    direction="maximize",
    study_name="rf_feat_eng",
    sampler=optuna.samplers.TPESampler(seed=config["general"]["seed"]),
    pruner=pruner,
)

[I 2024-05-10 16:35:20,107] A new study created in memory with name: rf_feat_eng


In [ ]:
warnings.filterwarnings('ignore') #ignore pandas warnings

study_feat_eng.optimize(
    objective_feat_eng, 
    n_trials=NUM_TRIALS,
    timeout = 20 * 60 * 60, # timeout after 8 hours
    show_progress_bar=True
)

  0%|          | 0/500 [00:00<?, ?it/s]

[I 2024-05-10 16:35:34,743] Trial 0 finished with value: 0.9636904761904762 and parameters: {'n_estimators': 437, 'max_depth': 48, 'min_samples_split': 24, 'min_samples_leaf': 20, 'criterion': 'gini'}. Best is trial 0 with value: 0.9636904761904762.
[I 2024-05-10 16:36:06,479] Trial 1 finished with value: 0.973037037037037 and parameters: {'n_estimators': 880, 'max_depth': 34, 'min_samples_split': 23, 'min_samples_leaf': 1, 'criterion': 'gini'}. Best is trial 1 with value: 0.973037037037037.
[I 2024-05-10 16:36:16,939] Trial 2 finished with value: 0.9666666666666667 and parameters: {'n_estimators': 263, 'max_depth': 17, 'min_samples_split': 11, 'min_samples_leaf': 17, 'criterion': 'log_loss'}. Best is trial 1 with value: 0.973037037037037.
[I 2024-05-10 16:36:26,265] Trial 3 finished with value: 0.9648809523809524 and parameters: {'n_estimators': 225, 'max_depth': 21, 'min_samples_split': 13, 'min_samples_leaf': 15, 'criterion': 'gini'}. Best is trial 1 with value: 0.973037037037037.
[

KeyboardInterrupt: 

In [ ]:
best_params = study_feat_eng.best_params
    
print(best_params)

if "random_forest_feat_eng" in config:
    config["random_forest_feat_eng"].update(best_params)
else:
    config["random_forest_feat_eng"] = best_params

# see: https://stackoverflow.com/questions/12470665/how-can-i-write-data-in-yaml-format-in-a-file
with open("classical_ml_config.yaml", "w") as file:
    yaml.dump(config, file, default_flow_style=False)

{'n_estimators': 680, 'max_depth': 43, 'min_samples_split': 8, 'min_samples_leaf': 1, 'criterion': 'gini'}


In [ ]:
p_importance_feat_eng = optuna.visualization.plot_param_importances(study_feat_eng)
p_importance_feat_eng.show()

In [ ]:
p_history_feat_eng = optuna.visualization.plot_optimization_history(study_feat_eng)
p_history_feat_eng.show()